# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/6953/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,996,Ingrid,-0.8293230493962478,0.6185177714634211
2000-01-01 00:00:01,1003,Hannah,-0.07128354536285775,0.24851354317467833
2000-01-01 00:00:02,1021,Zelda,-0.8968545169497966,-0.67324743817605
2000-01-01 00:00:03,1001,Quinn,0.8696536885733486,-0.3784484854470016
2000-01-01 00:00:04,971,Xavier,0.09593573566607638,0.009868686032291851
2000-01-01 00:00:05,1007,Quinn,-0.06914232351294936,0.557456138110302
2000-01-01 00:00:06,1046,Xavier,-0.2646264744203921,-0.4196441803663724
2000-01-01 00:00:07,1007,George,0.6195033290721437,-0.23488202664470204
2000-01-01 00:00:08,1012,Charlie,0.7215094836507356,0.22861584973466043


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,999,Zelda,-0.7769826926833201,0.2587718586835106
2000-01-30 00:00:01,1061,Tim,-0.18708459282738832,-0.021688993221798958
2000-01-30 00:00:02,998,Ray,-0.9528170515752221,-0.6763330986235065
2000-01-30 00:00:03,996,Edith,0.9170820523980618,0.12714410832349188
2000-01-30 00:00:04,989,Hannah,-0.4850149124849643,-0.8573785148973156
2000-01-30 00:00:05,1015,Quinn,-0.9652503656913327,0.5213793429876297
2000-01-30 00:00:06,1053,Wendy,0.6860204236845926,-0.2678631419406099
2000-01-30 00:00:07,957,Tim,-0.07177577716070771,-0.168878146494172
2000-01-30 00:00:08,936,Jerry,0.797672833481327,0.03223040244545272


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,996,Ingrid,-0.829323,0.618518
1,2000-01-01 00:00:01,1003,Hannah,-0.071284,0.248514
2,2000-01-01 00:00:02,1021,Zelda,-0.896855,-0.673247
3,2000-01-01 00:00:03,1001,Quinn,0.869654,-0.378448
4,2000-01-01 00:00:04,971,Xavier,0.095936,0.009869


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,996,Ingrid,-0.829323,0.618518
1,2000-01-01 00:00:01,1003,Hannah,-0.071284,0.248514
2,2000-01-01 00:00:02,1021,Zelda,-0.896855,-0.673247
3,2000-01-01 00:00:03,1001,Quinn,0.869654,-0.378448
4,2000-01-01 00:00:04,971,Xavier,0.095936,0.009869


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.46 s, sys: 430 ms, total: 3.89 s
Wall time: 3.37 s


name
Alice      -0.001677
Bob         0.003392
Charlie     0.000495
Dan         0.000406
Edith      -0.001494
Frank       0.003373
George      0.001867
Hannah     -0.000782
Ingrid     -0.001189
Jerry       0.001204
Kevin      -0.001649
Laura       0.001190
Michael     0.000921
Norbert     0.000037
Oliver      0.000733
Patricia    0.000364
Quinn       0.001329
Ray        -0.000844
Sarah       0.000968
Tim         0.001882
Ursula     -0.001604
Victor      0.000852
Wendy      -0.001178
Xavier      0.001089
Yvonne      0.001993
Zelda       0.002102
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.1 s, sys: 77.6 ms, total: 1.18 s
Wall time: 1.1 s


name
Alice      -0.001677
Bob         0.003392
Charlie     0.000495
Dan         0.000406
Edith      -0.001494
Frank       0.003373
George      0.001867
Hannah     -0.000782
Ingrid     -0.001189
Jerry       0.001204
Kevin      -0.001649
Laura       0.001190
Michael     0.000921
Norbert     0.000037
Oliver      0.000733
Patricia    0.000364
Quinn       0.001329
Ray        -0.000844
Sarah       0.000968
Tim         0.001882
Ursula     -0.001604
Victor      0.000852
Wendy      -0.001178
Xavier      0.001089
Yvonne      0.001993
Zelda       0.002102
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 994 ms, sys: 69.9 ms, total: 1.06 s
Wall time: 1.02 s


name
Alice      -0.001677
Bob         0.003392
Charlie     0.000495
Dan         0.000406
Edith      -0.001494
Frank       0.003373
George      0.001867
Hannah     -0.000782
Ingrid     -0.001189
Jerry       0.001204
Kevin      -0.001649
Laura       0.001190
Michael     0.000921
Norbert     0.000037
Oliver      0.000733
Patricia    0.000364
Quinn       0.001329
Ray        -0.000844
Sarah       0.000968
Tim         0.001882
Ursula     -0.001604
Victor      0.000852
Wendy      -0.001178
Xavier      0.001089
Yvonne      0.001993
Zelda       0.002102
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html